This notebook used ITIS TSNs discovered and cached in the Taxonomic Information Registry to look for information from the USFWS Threatened and Endangered Species System web service. It cached either a negative result or a set of key/value pairs from the TESS service of interest in characterizing species in the TIR.

In [14]:
import requests,configparser,re
from IPython.display import display

In [28]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [16]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [37]:
# Basic function to insert subject ID, property, and value into tircache
def cacheToTIR(gid,infotype,pairs):
    updateQ = "UPDATE tir.tir2 SET "+infotype+" = '"+pairs+"' WHERE gid = "+str(gid)
    return requests.get(getBaseURL()+"&q="+updateQ).json()

### Query TESS by TSN
There are several potential query points into TESS, but for now, we are just using the TSN-based search.

In [30]:
def queryTESS(tsn):
    tessSpeciesQueryByTSNBaseURL = "https://ecos.fws.gov/ecp0/TessQuery?request=query&xquery=/SPECIES_DETAIL[TSN="
    return requests.get(tessSpeciesQueryByTSNBaseURL+tsn+"]").text

### Package up the TESS data
This function works through the XML result from the TESS service. It will always return the data/time that the packaging step ran, the TSN that was used, and a result code. The result can be "none", "error" or "success". If successful, information is packaged from specific TESS attributes of interest into key/value pairs for later use.

In [24]:
def packageTESSPairs(tsn,tessData):
    import datetime
    from lxml import etree
    from io import StringIO
    dt = datetime.datetime.utcnow().isoformat()
    tessPairs = '"cacheDate"=>"'+dt+'"'
    tessPairs = tessPairs+',"tsn"=>"'+tsn+'"'

    if tessData.find('<results/>') > 0:
        tessPairs = tessPairs+',"result"=>"none"'
    else:
        try:
            rawXML = tessData.replace('<?xml version="1.0" encoding="iso-8859-1"?>', '')
            f = StringIO(rawXML)
            tree = etree.parse(f)
            tessPairs = tessPairs+',"result"=>"success"'
            tessPairs = tessPairs+',"entityId"=>"'+tree.xpath('/results/SPECIES_DETAIL/ENTITY_ID')[0].text+'"'
            tessPairs = tessPairs+',"SpeciesCode"=>"'+tree.xpath('/results/SPECIES_DETAIL/SPCODE')[0].text+'"'
            tessPairs = tessPairs+',"CommonName"=>"'+tree.xpath('/results/SPECIES_DETAIL/COMNAME')[0].text+'"'
            tessPairs = tessPairs+',"PopulationDescription"=>"'+tree.xpath('/results/SPECIES_DETAIL/POP_DESC')[0].text+'"'
            tessPairs = tessPairs+',"Status"=>"'+tree.xpath('/results/SPECIES_DETAIL/STATUS')[0].text+'"'
            tessPairs = tessPairs+',"StatusText"=>"'+tree.xpath('/results/SPECIES_DETAIL/STATUS_TEXT')[0].text+'"'
            rListingDate = tree.xpath('/results/SPECIES_DETAIL/LISTING_DATE')
            if len(rListingDate) > 0:
                tessPairs = tessPairs+',"ListingDate"=>"'+rListingDate[0].text+'"'
            tessPairs = tessPairs.replace("\'","''").replace(";","|").replace("--","-")
        except:
            tessPairs = tessPairs+',"result"=>"error"'

    return tessPairs


### Get TSNs for processing
Right now, we retrieve everything from the TIR table that has an ITIS TSN and does not currently have any TESS data. In future, we should come up with some other way of triggering this processing service and come up with a plan for periodic checks and updates.

In [33]:
tsns = requests.get(getBaseURL()+"&q=SELECT gid,itis->'discoveredTSN' AS discoveredtsn,itis->'acceptedTSN' AS acceptedtsn FROM tir.tir2 WHERE tess IS NULL AND itis->'itisMatchMethod' NOT LIKE 'NotMatched%' LIMIT 100").json()


### Process the records to query and return information from TESS
In this current process, I first check the discovered TSN from the ITIS cache. If that one does not return any results, I check the accepted TSN from that record (if it exists).

In [36]:
for feature in tsns["features"]:
    thisRecord = {}
    thisRecord["gid"] = feature["properties"]["gid"]
    thisRecord["discoveredTSN"] = feature["properties"]["discoveredtsn"]
    thisRecord["acceptedTSN"] = feature["properties"]["acceptedtsn"]
    
    # Query based on discovered TSN and package data
    thisRecord["tessPairs"] = packageTESSPairs(thisRecord["discoveredTSN"],queryTESS(thisRecord["discoveredTSN"]))

    if '"result"=>"none"' in thisRecord["tessPairs"] and type(thisRecord["acceptedTSN"]) is str:
        # Query based on discovered TSN and package data
        thisRecord["tessPairs"] = packageTESSPairs(thisRecord["acceptedTSN"],queryTESS(thisRecord["acceptedTSN"]))

    display (thisRecord)
    print (cacheToTIR(thisRecord["gid"],"tess",thisRecord["tessPairs"]))


{'acceptedTSN': None,
 'discoveredTSN': '173711',
 'gid': 9748,
 'tessPairs': '"cacheDate"=>"2017-05-07T16:38:27.345660","tsn"=>"173711","result"=>"success","entityId"=>"2423","SpeciesCode"=>"D01J","CommonName"=>"Techachapi slender salamander","PopulationDescription"=>"Wherever found","Status"=>"RT","StatusText"=>"Resolved Taxon"'}

{'_execution_time': 0.064, 'success': True, 'auth_check': {'auth_level': None, 'session': None, 'success': True}, 'affected_rows': 1}


{'acceptedTSN': None,
 'discoveredTSN': '22200',
 'gid': 16150,
 'tessPairs': '"cacheDate"=>"2017-05-07T16:38:28.823732","tsn"=>"22200","result"=>"success","entityId"=>"564","SpeciesCode"=>"Q109","CommonName"=>"Aupaka","PopulationDescription"=>"Wherever found","Status"=>"T","StatusText"=>"Threatened","ListingDate"=>"1996-10-10"'}

{'_execution_time': 0.063, 'success': True, 'auth_check': {'auth_level': None, 'session': None, 'success': True}, 'affected_rows': 1}


{'acceptedTSN': None,
 'discoveredTSN': '771341',
 'gid': 21,
 'tessPairs': '"cacheDate"=>"2017-05-07T16:38:30.346409","tsn"=>"771341","result"=>"none"'}

...

{'_execution_time': 0.058, 'success': True, 'auth_check': {'auth_level': None, 'session': None, 'success': True}, 'affected_rows': 1}


{'acceptedTSN': None,
 'discoveredTSN': '36399',
 'gid': 15604,
 'tessPairs': '"cacheDate"=>"2017-05-07T16:40:28.768503","tsn"=>"36399","result"=>"none"'}

{'_execution_time': 0.065, 'success': True, 'auth_check': {'auth_level': None, 'session': None, 'success': True}, 'affected_rows': 1}


{'acceptedTSN': None,
 'discoveredTSN': '175304',
 'gid': 41,
 'tessPairs': '"cacheDate"=>"2017-05-07T16:40:29.891921","tsn"=>"175304","result"=>"success","entityId"=>"9554","SpeciesCode"=>"B0G3","CommonName"=>"Sharp shinned hawk","PopulationDescription"=>"Wherever found","Status"=>"SC","StatusText"=>"Species of Concern"'}

{'_execution_time': 0.063, 'success': True, 'auth_check': {'auth_level': None, 'session': None, 'success': True}, 'affected_rows': 1}
